In [185]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import re
import unicodedata

In [8]:
restaurant_url = 'https://guide.michelin.com/en/california/oakland/restaurant/commis'

In [9]:
restaurant_page = requests.get(restaurant_url)

In [11]:
soup = BeautifulSoup(restaurant_page.text, 'html.parser')

In [12]:
soup


<!DOCTYPE html>

<html class="full-screen-mobile" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0, user-scalable=0" name="viewport"/>
<meta content="" name="author"/>
<meta content="#bd2333" name="theme-color"/>
<meta content="vFUp7XI3G-xKLevkT2qftuscMInOxirdCu0oYr7_uLM" name="google-site-verification">
<meta content="eZwPBJ-0r-9qysyJy_aEPti-m_vr-h_oUJy2qUR1144" name="google-site-verification">
<meta content="https://d3h1lg3ksw6i6b.cloudfront.net/guide/xlarge/259803_1.jpg" itemprop="image">
<meta content="Article" property="og:type">
<meta content="MICHELIN Guide" property="og:site_name"/>
<meta content="https://guide.michelin.com/en/california/oakland/restaurant/commis" property="og:url"/>
<meta content="https://d3h1lg3ksw6i6b.cloudfront.net/guide/xlarge/259803_1.jpg" property="og:image"/>
<meta content="Commis – Oakland - a MICHELIN Guide Restaurant" itemprop="name"/>
<meta content="Commis – Oakland - a MICHELIN Guide Restaurant" proper

In [16]:
soup.select('title')

[<title>Commis – Oakland - a MICHELIN Guide Restaurant</title>,
 <title>MICHELIN Guide - the official website</title>]

In [190]:
star_dict = {
    'm': 1,
    'n': 2,
    'o': 3
}

utensil_dict = {
    2: 0 #simple shop
    'ò': 1 #Comfortable
    'ó': 2 #Quite Comfortable
    'ô': 3 #Very Comfortable
    'õ': 4 #Top class comfortable
    'ö': 5 #Luxury in the traditional style
}


def num_stars(site):
    content = site.find('i', 'fa-michelin restaurant-details__classification--list-icon color-primary').contents
    letter = content[0]
    return star_dict[letter] 

In [191]:
def get_address(site):
    content = soup.find('ul', 'restaurant-details__heading--list').contents
    inner = res_heading[1].contents
    
    return inner[1] 
    

In [139]:
def remove_space_lines(text):
    pattern1 = re.compile(r'[\s\s+]')
    return re.sub(pattern1, '', text)

In [157]:
def split_money(text):
    currency = text[-3:]
    
    money = text[:-3]    
    money = money.split('-')
    
    min_price = money[0]
    max_price = money[1]
    
    money_series = pd.Series({'min_price': min_price, 'max_price': max_price, 'currency': currency})
    
    return money_series 

In [173]:
def get_price_cuisine(site):
    content = soup.find('li', 'restaurant-details__heading-price').contents[0]
    content = content.strip()
    content = remove_space_lines(content)
    
    content = content.split('•')
    
    money_series = split_money(content[0])
    
    cuisine = pd.Series(content[1], index = ['cuisine'])
    
    money_series = money_series.append(cuisine)
    
    return money_series
    
    


In [174]:
get_price_cuisine(soup)

min_price              75
max_price             150
currency              USD
cuisine      Contemporary
dtype: object

In [188]:
def get_description(site):
    contents = soup.find('div', 'js-show-description-text').contents[1].contents[0]
    return unicodedata.normalize('NFKD', contents)




In [ ]:
def get_comfort(site):
    contents = 

In [201]:
a = soup.find('i', "fa-michelin restaurant-details__classification--list-icon color-black")